In [20]:
const course_name = "청석공원 파크골프장"
const address = "광주시 경안동 242번지"
const hole_count = 18
const slug = course_name.replace(/ /g, "-").toLowerCase()
const opening_hours = "09:00 ~ 17:00 / 동절기 휴장"
const regular_closed_days = undefined
const website = "https://www.gumc.or.kr/business/park2"
const registration_method = "광주시민 및 관내종사자만 사용 가능"
const reference = "https://www.gumc.or.kr/business/park2"
const phone_number = undefined;


golf_course, road_address, address, contact, operation 테이블 생성해야 함

# 주소 변환

In [ ]:
import fetch from 'npm:node-fetch';

const query = address;
const url = `https://dapi.kakao.com/v2/local/search/address.json?query=${encodeURIComponent(query)}`;

const response = await fetch(url, {
    method: 'GET',
    headers: {
        'Authorization': `KakaoAK 6f447bc34a2d339805e6816490bf2a66`
    }
})
const json = await response.json();
const { documents } = json;
const address = documents[0].address;
const address_name = address ? address.address_name : "";
const road_address = documents[0].road_address || undefined;
const road_address_name = road_address ? ` ${road_address.address_name}`  : "";
const searchable_address =  `${address_name}${road_address_name} ${course_name}`.trim()

## Create Client

In [ ]:
import { createClient } from 'https://esm.sh/@supabase/supabase-js@2'


// Create a single supabase client for interacting with your database
// add Client Key
const supabase = createClient() 

## Create Golf Course

In [ ]:
const golfCourseParams = {
  name: course_name,
  searchable_address,
  hole_count: hole_count,
  slug,
}

In [ ]:
const { data: golfCourse, error: golfCourseError } = await supabase
.from('golf_course')
.upsert(golfCourseParams)
.select()
.single()
console.log(golfCourse, golfCourseError)

## Create Address, Road Address


In [13]:
const {data: addressResponse, error: addressError} = await supabase
.from("address")
.upsert({
  ...address,
  golf_course_id: golfCourse.id
})
.select()
console.log(addressResponse, addressError)

[
  {
    id: 769,
    address_name: "경기 광주시 경안동 242",
    b_code: "4161010100",
    h_code: "4161051000",
    main_address_no: "242",
    mountain_yn: "N",
    region_1depth_name: "경기",
    region_2depth_name: "광주시",
    region_3depth_h_name: "경안동",
    region_3depth_name: "경안동",
    sub_address_no: "",
    x: 127.264963684938,
    y: 37.4120694738893,
    golf_course_id: 368
  }
] null


In [12]:
if(road_address){
const {data: roadAddressResponse, error: roadAddressError} = await supabase
.from("road_address")
.upsert({
  ...road_address,
  golf_course_id: golfCourse.id
})
.select()
console.log(roadAddressResponse, roadAddressError)
}

## Operation

In [17]:
const operationParams = {
  regular_closed_days,
  opening_hours,
  website,
  registration_method,
  reference,
  golf_course_id: golfCourse.id
}

In [18]:
const {data: operationData, error: operationError} = await supabase
.from("operation")
.upsert({
  ...operationParams,
})
.select()

## Contact

In [21]:
const contactParams = {
  phone_number,
  golf_course_id: golfCourse.id
}

In [22]:
const {data: contactData, error: contactError} = await supabase
.from("contact")
.upsert({
  ...contactParams,
})
.select()